### Import Dependencies

In [2]:
!pip install newspaper3k
!pip install --upgrade pip

In [3]:
import os
import pandas as pd
import plotly.graph_objects as go
import random
import newspaper
from newspaper import Article, fulltext, Config, ArticleException
import time
import numpy as np
import requests
import re
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import threading

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Define Required Functions

### FY24 ###


This function takes a string argument 'country' and returns a string representing the region the country belongs to.

The regions are defined as follows:

- 'Brazil' for Brazil
- 'Argentina' for Argentina
- 'Mexico' for Mexico
- 'Chile' for Chile
- 'Andean Region' for Colombia, Bolivia, Ecuador, Paraguay, Peru and Uruguay
- 'Central' for Central America countries, Costa Rica, Guatemala, Honduras, Panama, El Salvador, Dominican Republic, Republica Dominicana, Jamaica, Puerto Rico, Trinidad y Tobago and Trinidad and Tobago
- 'Other' for any other country


### COLORS: HEX CODES
- AMAZON : #6B007B
- GOOGLE : #D9B300
- MICROSOFT : #6BA5D9
- SALESFORCE : #AEAEAE

In [6]:
def get_region(country):
    if country == 'Brazil':
        return 'Brazil'
    elif country == 'Argentina':
        #return 'Hispanic South America'
        return 'Spanish South America'
    elif country == 'Mexico':
        return 'Mexico'
    elif country == 'Chile':
        #return 'Hispanic South America'
        return 'Spanish South America'    
    elif country in ['Colombia', 'Bolivia', 'Ecuador', 'Paraguay', 'Peru', 'Uruguay', 'Perú']:
        #return 'Hispanic South America'
        return 'Spanish South America'
    elif country in ['Central', 'Costa Rica', 'Guatemala', 'Honduras', 'Panama', 'El Salvador', 'Dominican Republic', 'Republica Dominicana', 'Jamaica', 'Puerto Rico', 'Trinidad y Tobago', 'Trinidad and Tobago']:
        #return 'Central'
        return 'Central Caribbean'
    else:
        return 'Other'

In [49]:
user = 'katia.bedolla@porternovelli.mx'
API_TOKEN = '7a7668b8111a6e4d5750c12a8c93b56d'

class DiffbotClient(object):
    base_url = 'http://api.diffbot.com/'

    def __init__(self):
        self.session = requests.Session()

    def request(self, url, token, api, fields=None, version=3, **kwargs):
        params = {"url": url, "token": token}
        if fields:
            params['fields'] = fields
        params.update(kwargs)
        response = self.session.get(self.compose_url(api, version), params=params, timeout=10)
        response.raise_for_status()
        return response.json()

    def compose_url(self, api, version_number):
        version = self.format_version_string(version_number)
        return '{}{}/{}'.format(self.base_url, version, api)

    @staticmethod
    def format_version_string(version_number):
        return 'v{}'.format(version_number)

def get_content_diffbot(url):
    diffbot = DiffbotClient()
    token = API_TOKEN
    api = "analyze"
    try:
        response = diffbot.request(url, token, api)
        if 'objects' in response and len(response['objects']) > 0 and 'text' in response['objects'][0]:
            return response['objects'][0]['text']
        else:
            return "No Content"
    except:
        return "Something went wrong with url"


def extract_content_from_urls(urls):
    threads = []
    contents = []
    MAX_CONCURRENCY = 15  # set the maximum number of concurrent threads
    for i in range(0, len(urls), MAX_CONCURRENCY):
        batch = urls[i:i+MAX_CONCURRENCY]
        for url in batch:
            thread = threading.Thread(target=get_content_news, args=(url, contents))
            thread.start()
            threads.append(thread)
        
        for thread in threads:
            thread.join()
        
    return contents

def get_content_news(url, contents):
    user_agent_list = [
   #Chrome
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',

    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]
    user_agent_list = random.choice(user_agent_list)
    user_agent = random.choice(user_agent_list)
    config = Config()
    config.browser_user_agent = user_agent
    try:
        a = Article(url, config=config)
        a.download()
        a.parse()
        paragraphs = a.text
        paragraphs = re.sub(r'<a href=.+?(?=)>|<br/>|\\xa0|\n|</a>|\xa0|<strong>|</strong>|<br/>•|<i(.*?)</i>|<img(.*?)>','',str(paragraphs))

        if isinstance(paragraphs, str) and len(paragraphs) > 0:
            print("URL Content from {} is correct".format(url))
            return paragraphs
        else:
            try:
                ext_diff = get_content_diffbot(url)
                print("URL Content from {} is correct from diffbot")
                return ext_diff
            except Exception as exce:
                print("URL Content from {} is OtherError")
    except ArticleException as e:
        if "Invalid leading whitespace, reserved character(s), or returncharacter(s) in header value: ' '" in str(e):
            print("Error: Invalid header value in article download.")
        elif "HTTPSConnectionPool(host='tecnoblog.net', port=443): Read timed out." in str(e):
            print("Error: Connection timed out while downloading article.")
        elif "403 Client Error: Forbidden" in str(e):
            print("Error: Forbidden to access the article.")
        elif "429 Client Error: Too Many Requests" in str(e):
            print("Error: Too Many Requests, waiting for 5 seconds before trying again.")
            sleep(5)
        elif "Article `download()` failed with HTTPSConnectionPool(host='geekzilla.tech', port=443): Read timed out. (read timeout=7) on URL https://geekzilla.tech/llega-red-hat-ansible-automation-platform-a-google-cloud/" in str(e):
            try:
                ext_diff = get_content_diffbot(url)
                print("URL Content from {} is correct from diffbot".format(url))
                return ext_diff
            except Exception as exce:
                print("URL Content from {} is OtherError")
        else:
            pass

# Define a function to process a single URL
def process_url(url):
    return get_content_news(url, [])

# This function is used to process URLs directly and extract the summary.
def process_url_summary(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        summary = article.summary
        summary = summary.replace("\n", " ")
        print(f'Summary for URL: {url} - Successful')
        return summary
    except:
        print(f'Summary for URL: {url} - Failed')
        return None

In [50]:
def latam_categories(path):
    companies_dfs = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            platform = file.split("_")[0].capitalize()
            df = pd.read_csv(os.path.join(path, file), sep= None, engine= 'python')
            
            # Add the Company and Category columns
            df['Company'] = file.split('_', 1)[0]
            df['Category'] = file.split('_', 1)[1].split('_', 1)[0]
            
            companies_dfs.append(df)

    return pd.concat(companies_dfs, ignore_index=True)


## AI - LATAM

In [51]:
ai = latam_categories("categories_latam/ai")
ai['Subsidiary']= ai['Country'].apply(get_region)
ai['Category']= 'AI'
ai['Region']= 'LATAM'
print(ai.shape)
ai.head(3)

(2574, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c17a11aa3fc98c1024cae20f080c6ee8fbd10464d844af...,2023-08-17 13:29:00,IA es un “maremoto” tan grande como internet: ...,https://www.elespectador.com/tecnologia/ia-es-...,elespectador.com,unknown journalist,Colombia,es,329107,3,2,Google,AI,Spanish South America,LATAM
1,7bd7c72adaebfe21a2f49154e2bca8a5b8e04610cf3e28...,2023-08-18 09:00:00,IA Generativa: Google lanza una nueva herramie...,https://www.iproup.com/innovacion/42300-ia-gen...,iproup.com,unknown journalist,Argentina,es,7552,11,2,Google,AI,Spanish South America,LATAM
2,2f09d981e4dfd063a3bd152502768bfae605d08a5c6deb...,2023-08-17 18:35:00,Número de campanhas políticas manipuladas usan...,https://olhardigital.com.br/2023/08/17/pro/num...,olhardigital.com.br,Vitoria Lopes Gomez,Brazil,pt,209607,2,2,Google,AI,Brazil,LATAM


## Brand Innovation - LATAM

In [52]:
brand_innovation = latam_categories("categories_latam/brand_innovation")
brand_innovation['Subsidiary']= brand_innovation['Country'].apply(get_region)
brand_innovation['Category']= 'Brand Innovation'
brand_innovation['Region']= 'LATAM'
print(brand_innovation.shape)
brand_innovation.head()

(1075, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,df0cdb500cf33aa934bb67171cbedfb53e6263598da792...,2023-07-13 14:01:12,Como usar o Google Bard no Brasil? Confira o p...,https://www.tecmundo.com.br/software/266315-co...,tecmundo.com.br,Carlos Palmeira,Brazil,pt,544623,18,2,Google,Brand Innovation,Brazil,LATAM
1,404fa1fa6b6239f27e9ffbac1a33be84341a65b50a149b...,2023-07-18 03:09:00,Esta firma de Wall Street explicó por qué las ...,https://www.forbesargentina.com/money/esta-fir...,forbesargentina.com,Journalist Unknown,Argentina,es,14777,4,2,Google,Brand Innovation,Spanish South America,LATAM
2,fb8683ae06887b63efd7245701ab69fce55dd12af77e50...,2023-07-17 07:20:00,Samsung prueba a introducir ChatGPT en el nave...,https://www.lanacion.com.ar/agencias/samsung-p...,lanacion.com.ar,unknown journalist,Argentina,es,998020,0,1,Google,Brand Innovation,Spanish South America,LATAM
3,37bca161f8f04063d120b48cc6d301efb60fe85a16b295...,2023-07-26 07:10:00,La nueva experiencia de búsqueda de Google bas...,https://www.lanacion.com.ar/agencias/la-nueva-...,lanacion.com.ar,unknown journalist,Argentina,es,1095407,0,2,Google,Brand Innovation,Spanish South America,LATAM
4,295e49c072c7eaf1ac0a5016cacea5c6327d7a050ba046...,2023-07-21 14:52:00,"Na corrida por chips de IA, Google descobre se...",https://olhardigital.com.br/2023/07/21/pro/na-...,olhardigital.com.br,Pedro Borges Spadoni,Brazil,pt,199756,58,2,Google,Brand Innovation,Brazil,LATAM


## Business Applications - LATAM

In [53]:
business_applications = latam_categories("categories_latam/business_applications")
business_applications['Subsidiary']= business_applications['Country'].apply(get_region)
business_applications['Category']= 'Business Applications'
business_applications['Region']= 'LATAM'
print(business_applications.shape)
business_applications.head()

(62, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,807201a4bb672573156b743f0457dad468ea8e1ff23d05...,2023-07-26 17:40:00,"De la mano de la Inteligencia Artificial, tecn...",https://ct.moreover.com/?a=51395965739&p=8vr&v...,iprofesional.com,Adrián Moreno,Argentina,es,155848,2,2,Microsoft,Business Applications,Spanish South America,LATAM
1,b91ece142bf64d6baf4df41d5b2622c1aedcb76789dd86...,2023-09-29 09:42:00,"Caixa, Elo e Microsoft fazem primeira compra e...",https://istoedinheiro.com.br/caixa-elo-e-micro...,istoedinheiro.com.br,Estadão Conteúdo,Brazil,pt,122995,0,1,Microsoft,Business Applications,Brazil,LATAM
2,12db19c9e3075ab33acf2d97dcaf0879c0170d8a20804c...,2023-07-03 11:30:00,5 tendências em alta de colaboração no local d...,https://itforum.com.br/computerworld/5-tendenc...,itforum.com.br,Matthew Finnegan,Brazil,pt,19579,16,2,Microsoft,Business Applications,Brazil,LATAM
3,2d19767e488b65ce71afa2d0a385d2bec75e65de995226...,2023-09-10 10:03:00,Zoom presentó nuevas herramientas de IA Genera...,https://www.forbesargentina.com/innovacion/zoo...,forbesargentina.com,Melody Brue,Argentina,es,18077,5,2,Microsoft,Business Applications,Spanish South America,LATAM
4,c5e03fe62b0b6037e80d8bbb12648b73055840a180d3cc...,2023-09-29 10:57:00,"Caixa, Elo e Microsoft fazem 1ª compra de títu...",https://exame.com/future-of-money/caixa-elo-mi...,exame.com,João Pedro Malar,Brazil,pt,277677,33,1,Microsoft,Business Applications,Brazil,LATAM


## Cloud Platform - LATAM

In [55]:
cloud_platform = latam_categories("categories_latam/cloud_platform")
cloud_platform['Subsidiary']= cloud_platform['Country'].apply(get_region)
cloud_platform['Category']= 'Cloud Platform'
cloud_platform['Region']= 'LATAM'
print(cloud_platform.shape)
cloud_platform.head()

(697, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,686e32410de6e17e4eabecc3dfdf4065133c0c96211373...,2023-07-25 17:50:00,alphabet reporta ganancias superiores a las pr...,https://www.lanacion.com.ar/agencias/alphabet-...,lanacion.com.ar,unknown journalist,Argentina,es,1095407,0,2,Google,Cloud Platform,Spanish South America,LATAM
1,5315d0bfa702b7d87423f1b168afd1366660e3f859349c...,2023-07-26 16:15:37,Google sube 6% en Bolsa por publicidad digital...,https://expansion.mx/mercados/2023/07/26/accio...,expansion.mx,Staff Writer,Mexico,es,163254,3,2,Google,Cloud Platform,Mexico,LATAM
2,e36c6851f319f8ee443a1b183403ca4b7f9cffb8c45d66...,2023-08-29 04:59:00,Inteligencia artificial podrá cubrirte en tus ...,https://www.milenio.com/negocios/inteligencia-...,milenio.com,unknown journalist,Mexico,es,615997,2,2,Google,Cloud Platform,Mexico,LATAM
3,f07f7b6957736cfb7a231320c79b6d10418a7dcd6fc76f...,2023-08-31 00:07:00,Google Cloud establecerá operaciones en El Sal...,https://dplnews.com/google-cloud-establecera-o...,dplnews.com,Dinorah Navarro,Mexico,es,44797,2,2,Google,Cloud Platform,Mexico,LATAM
4,1b0e55bcbaca121fa411fda76dedae713924112d5bff42...,2023-09-04 12:24:00,Continental vai integrar serviços do Google Cl...,http://forbes.com.br/forbes-tech/2023/09/conti...,forbes.com.br,Fernanda Almeida,Brazil,pt,55801,2,2,Google,Cloud Platform,Brazil,LATAM


## Developers - LATAM

In [57]:
developers = latam_categories("categories_latam/developers")
developers['Subsidiary']= developers['Country'].apply(get_region)
developers['Region']= 'LATAM'
print(developers.shape)
developers.head()

(157, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0,2,Amazon,Developers,Spanish South America,LATAM
1,59e57a1ad35c6f5f24716ab1a289e853185e83dff80c54...,2023-08-01 20:16:00,Acumulando polvo y datos: cómo las aspiradoras...,https://portalinnova.cl/acumulando-polvo-y-dat...,portalinnova.cl,unknown journalist,Chile,es,4672,2,2,Amazon,Developers,Spanish South America,LATAM
2,aa4e6c002b7b27a80321f9aff89763c5b59a4a4a35ca48...,2023-09-28 14:00:00,AWS anuncia novas ofertas para acelerar a adoç...,https://inforchannel.com.br/2023/09/28/aws-anu...,inforchannel.com.br,unknown journalist,Brazil,pt,8194,0,2,Amazon,Developers,Brazil,LATAM
3,162bb80c94fdcec99946240b58f83d73635ef3ca833af2...,2023-07-30 05:13:00,Cómo aprender sobre IA generativa con estos si...,https://www.forbesargentina.com/innovacion/com...,forbesargentina.com,Nicolás Della Vecchia,Argentina,es,18077,9,2,Amazon,Developers,Spanish South America,LATAM
4,6ff33081a70ee1e3287ce37ce75486761158a64d864390...,2023-09-29 08:25:00,AWS anuncia nuevos modelos fundacionales y cap...,https://www.eltiempo.com/tecnosfera/novedades-...,eltiempo.com,unknown journalist,Colombia,es,726205,8,2,Amazon,Developers,Spanish South America,LATAM


## Productivity - LATAM

In [58]:
productivity = latam_categories("categories_latam/productivity")
productivity['Subsidiary']= productivity['Country'].apply(get_region)
productivity['Region']= 'LATAM'
print(productivity.shape)
productivity.head()

(832, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,2700d0349a578c6fac40a53295277d0491224e581afc89...,2023-07-03 07:00:00,Microsoft enfrenta otra investigación antimono...,https://expansion.mx/tecnologia/2023/07/03/mic...,expansion.mx,unknown journalist,Mexico,es,163254,3,2,Salesforce,Productivity,Mexico,LATAM
1,4ba5602bb70343afe867963803eed3089d1808454627f0...,2023-07-26 01:01:00,"Lidiane Jones: “Se você sempre estiver certo, ...",https://epocanegocios.globo.com/personalidades...,epocanegocios.globo.com,Andressa Rovani,Brazil,pt,132427,5,2,Salesforce,Productivity,Brazil,LATAM
2,bb033133770d5abc888ed5e21a5ae08a4b22d7599780c0...,2023-08-09 09:50:00,Repaginada no Slack deixa interface mais 'clea...,https://olhardigital.com.br/2023/08/09/reviews...,olhardigital.com.br,Borges Spadoni,Brazil,pt,209607,1,2,Salesforce,Productivity,Brazil,LATAM
3,80fed41c388cc62635681e38cf79d814bbe3d5973f2167...,2023-09-21 12:18:00,"IA tornará o trabalho mais humano, diz CEO do ...",https://olhardigital.com.br/2023/09/21/pro/ia-...,olhardigital.com.br,Alessandro Di Lorenzo,Brazil,pt,209607,20,2,Salesforce,Productivity,Brazil,LATAM
4,52d23dc11c0d1dd75c4b73a7865ae4bde6c449efbb4b9f...,2023-08-02 13:00:00,Salesforce lança Slack Sales Elevate para ajud...,https://inforchannel.com.br/2023/08/02/salesfo...,inforchannel.com.br,unknown journalist,Brazil,pt,8194,0,2,Salesforce,Productivity,Brazil,LATAM


## Security - LATAM

In [59]:
security = latam_categories("categories_latam/security")
security['Subsidiary']= security['Country'].apply(get_region)
security['Region']= 'LATAM'
print(security.shape)
security.head()

(895, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,d1412badf223691ee0317d2f3f69d1f99096482de44bad...,2023-08-08 14:18:00,Android 14 terá melhorias de segurança “inédit...,https://canaltech.com.br/android/android-14-te...,canaltech.com.br,Felipe Demartini,Brazil,pt,338531,7,2,Google,Security,Brazil,LATAM
1,aaffdfbc0dc5d5953771151f89a648ee2f5f38dfca32a5...,2023-08-16 02:00:00,Google Drive: Cómo proteger y encriptar los da...,https://www.estrategiaynegocios.net/tecnologia...,estrategiaynegocios.net,unknown journalist,El Salvador,es,26019,3,2,Google,Security,Central Caribbean,LATAM
2,74b032edc0a419884b4ebc65a047ff1b1b4c71a0e40241...,2023-08-30 09:20:00,Con este trucazo podrá saber si su correo de G...,https://www.semana.com/tecnologia/articulo/con...,semana.com,unknown journalist,Colombia,es,291733,3,2,Google,Security,Spanish South America,LATAM
3,2599c6fa3bb0b4c292fdd50d3826782f6c7fa244912723...,2023-09-08 14:41:43,"A qué se dedican los hackers rojos, los guardi...",https://www.infobae.com/tecno/2023/09/08/a-que...,infobae.com,unknown journalist,Argentina,es,1529212,37,2,Google,Security,Spanish South America,LATAM
4,47dc98df86b3abbd8a70ecb619d0cc8a5a8366b1536bba...,2023-09-13 19:09:58,"Falha de segurança afeta Chrome, Firefox, Edge...",https://tecnoblog.net/noticias/2023/09/13/falh...,tecnoblog.net,Giovanni Santa Rosa,Brazil,pt,175722,28,2,Google,Security,Brazil,LATAM


## Sustainability - LATAM

In [60]:
sustainability = latam_categories("categories_latam/sustainability")
sustainability['Subsidiary']= sustainability['Country'].apply(get_region)
sustainability['Region']= 'LATAM'
print(sustainability.shape)
sustainability.head()

(105, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,31dbb5315a47f03ea2829385974cdeab5549ed5cd7d5f6...,2023-09-05 13:00:00,Microsoft registra patente de mochila intelige...,https://epocanegocios.globo.com/tecnologia/not...,epocanegocios.globo.com,Soraia Alves,Brazil,pt,132427,1,2,Microsoft,Sustainability,Brazil,LATAM
1,365a4b427fc0ab2b0f74397ebe6664d9cf7c3c384c2355...,2023-08-31 12:12:07,Microsoft busca ideas innovadoras para protege...,https://www.elnuevodia.com/brandstudio-publish...,elnuevodia.com,unknown journalist,Puerto Rico,es,264700,45,2,Microsoft,Sustainability,Central Caribbean,LATAM
2,e219835df632d818a7c194d8a0ebc2a70afb7c17fef9c3...,2023-07-10 07:00:00,Microsoft y sus ambiciosas metas en materia de...,https://impactotic.co/microsoft-y-sus-metas-de...,impactotic.co,Daniela Jaramillo Castillo,Colombia,es,8951,17,2,Microsoft,Sustainability,Spanish South America,LATAM
3,2ea9ab60d7255e7ac874817a972326d6a595c1e8fe8477...,2023-09-04 09:43:59,A inteligência artificial vai piorar as mudanç...,https://www.estadao.com.br/sustentabilidade/a-...,estadao.com.br,unknown journalist,Brazil,pt,597225,19,2,Microsoft,Sustainability,Brazil,LATAM
4,7af3d70bc23291faa0ebfcbc9de28a7e3c8a6751ca1731...,2023-09-28 21:37:00,Microsoft busca concretos mais sustentáveis pa...,https://olhardigital.com.br/2023/09/28/pro/mic...,olhardigital.com.br,Rodrigo Mozelli,Brazil,pt,209607,1,2,Microsoft,Sustainability,Brazil,LATAM


## Trust Overview - LATAM

In [61]:
trust_overview = latam_categories("categories_latam/trust_overview")
trust_overview['Subsidiary']= trust_overview['Country'].apply(get_region)
trust_overview['Category']= 'Trust Overview'
trust_overview['Region']= 'LATAM'
print(trust_overview.shape)
trust_overview.head()

(3883, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,9b783b78c5773153e76592fbdcd3fe34bbe3e2610da10c...,2023-09-11 09:30:00,Focos inteligentes compatibles con Alexa: con ...,https://www.xataka.com.mx/seleccion/focos-inte...,xataka.com.mx,Alejandro Aranda,Mexico,es,58968,0.0,2,Amazon,Trust Overview,Mexico,LATAM
1,53ee0884a61d9254f495c9830de47a7a1abaecf7068689...,2023-09-15 18:00:00,¡Oficialmente llega a México el programa educa...,https://geekzilla.tech/oficialmente-llega-a-me...,geekzilla.tech,unknown journalist,Mexico,es,2731,0.0,2,Amazon,Trust Overview,Mexico,LATAM
2,71d72de811e5d0d1df925547d77dd05f650ff7f3ecfd66...,2023-09-26 11:48:00,"Crítica das Big Tech, presidente da Comissão d...",https://epocanegocios.globo.com/reuters/notici...,epocanegocios.globo.com,Unknow Jornalist,Brazil,pt,132427,2.0,2,Amazon,Trust Overview,Brazil,LATAM
3,289fc1b7e4ea163d569ff0f533145cf7b2ebea49d55af4...,2023-09-25 05:39:00,Amazon investe até US$ 4 bilhões em empresa de IA,https://exame.com/inteligencia-artificial/amaz...,exame.com,unknown journalist,Brazil,pt,277677,2.0,2,Amazon,Trust Overview,Brazil,LATAM
4,7a33e5e4067e5770be1abf52b17ff3b22b12e82ffef3f1...,2023-09-26 14:21:00,"Crítica das Big Tech, presidente da Comissão d...",https://www.uol.com.br/tilt/noticias/reuters/2...,uol.com.br,David Shepardson,Brazil,pt,4794257,0.0,2,Amazon,Trust Overview,Brazil,LATAM


## AI - Countries

In [64]:
ai_countries = latam_categories("categories_countries/ai")
ai_countries['Subsidiary']= ai_countries['Country'].apply(get_region)
ai_countries['Category']= 'AI'
ai_countries['Region']= 'COUNTRIES'
print(ai_countries.shape)
ai_countries.head(3)

(2878, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,c8c49005158027f58df3198819e503989430d7ee16021f...,2023-07-27 07:23:00,AWS lanza siete nuevos cursos para aprender IA...,https://portalinnova.cl/aws-lanza-siete-nuevos...,portalinnova.cl,unknown journalist,Chile,es,4672,0,2,Amazon,AI,Spanish South America,COUNTRIES
1,bac1b1365d3347b0c72135e102e11a7ead5b378fc3f26a...,2023-09-20 19:15:53,"Alexa ""casi humana"": Amazon presenta nueva ver...",https://www.df.cl/internacional/multinacionale...,df.cl,unknown journalist,Chile,es,45232,16,2,Amazon,AI,Spanish South America,COUNTRIES
2,9fadbc44ddb0af58e501328f0255e25a04a89ab5d778dc...,2023-09-06 08:16:00,Grupo BMW selecciona a AWS para impulsar la pl...,https://portalinnova.cl/grupo-bmw-selecciona-a...,portalinnova.cl,Journalist Unknown,Chile,es,4672,0,2,Amazon,AI,Spanish South America,COUNTRIES


## Brand Innovation - Countries

In [65]:
brand_innovation_countries = latam_categories("categories_countries/brand_innovation")
brand_innovation_countries['Subsidiary']= brand_innovation_countries['Country'].apply(get_region)
brand_innovation_countries['Category']= 'Brand Innovation'
brand_innovation_countries['Region']= 'COUNTRIES'
print(brand_innovation_countries.shape)
brand_innovation_countries.head(3)

(1401, 15)


,"﻿""Article ID""",Published date,Headline,Url,Outlet Name,Author Name,Country,Language,Impressions,Shares,Coded,Company,Category,Subsidiary,Region
0,1375b86f1ac844308c2cf0fc62eaa5674a7e5f5f17fdc1...,2023-08-30 10:51:00,7 aplicaciones para hacer traducción en tiempo...,https://www.lanacion.com.ar/tecnologia/7-aplic...,lanacion.com.ar,Desirée Jaimovich,Argentina,es,1095407,55,2,Google,Brand Innovation,Spanish South America,COUNTRIES
1,a1936cd57045b9a503305cf6b3bcfe319a1312249f7268...,2023-07-09 23:09:47,Las películas más populares en Google Colombia,https://www.infobae.com/noticias/2023/07/10/la...,infobae.com,Newsroom Infobae,Argentina,es,1442272,0,2,Google,Brand Innovation,Spanish South America,COUNTRIES
2,61e8a16e19535035d2e2128b81d4940d1306a3498c4143...,2023-08-15 15:32:59,Google Slides: cómo usar gratis esta herramien...,https://www.infobae.com/tecno/2023/08/15/googl...,infobae.com,Jimmy Pepinosa,Argentina,es,1529212,7,2,Google,Brand Innovation,Spanish South America,COUNTRIES
